In [5]:
import sys
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.master("local[1]").appName("spark-app-version-x").getOrCreate()

In [7]:
local_file = r'C:\Users\Yunis\Desktop\yellow_tripdata_2024-01.parquet'
df=spark.read.parquet(local_file)
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [8]:
df.select('VendorID','tpep_pickup_datetime','total_amount').where('total_amount>1').show(n=10)

+--------+--------------------+------------+
|VendorID|tpep_pickup_datetime|total_amount|
+--------+--------------------+------------+
|       2| 2024-01-01 00:57:55|        22.7|
|       1| 2024-01-01 00:03:00|       18.75|
|       1| 2024-01-01 00:17:06|        31.3|
|       1| 2024-01-01 00:36:38|        17.0|
|       1| 2024-01-01 00:46:51|        16.1|
|       1| 2024-01-01 00:54:08|        41.5|
|       2| 2024-01-01 00:49:44|       64.95|
|       1| 2024-01-01 00:30:40|        30.4|
|       2| 2024-01-01 00:26:01|        36.0|
|       2| 2024-01-01 00:28:08|         8.0|
+--------+--------------------+------------+
only showing top 10 rows



In [9]:
df.createOrReplaceTempView('yun')

In [10]:
spark.sql("select * from yun where total_amount>3").show(n=5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-01-01 00:57:55|  2024-01-01 01:17:43|              1|         1.72|         1|                 N|         186|          79|           2|       17.7|  1.0|    0.5|       0.

In [11]:
spark.stop()

In [12]:
import logging
from pyspark.sql import SparkSession

In [13]:
def rdd_to_dataframe(data, schema):
    """
    Example: This fn creates a Spark RDD, loads it into a Spark DataFrame, and returns the DataFrame 
    """
        
    # Create a SparkSession
    spark = SparkSession.builder.appName("RDDToDataFrame").getOrCreate()

    try:
        # Create an RDD from the input data, using Spark Context not Session!
        rdd = spark.sparkContext.parallelize(data)

        # Convert RDD to DataFrame
        df = spark.createDataFrame(rdd, schema)

        # Return the DataFrame, without stopping the SparkSession
        return df

    except Exception as e:
        # Log error and Stop the SparkSession
        logging.error('Error while transforming RDD to DF: {}'.format(e))
        spark.stop()


In [14]:
dept_data = [(1,"Big Data"), (2, "Finance"), (3,"Marketing")]
dept_schema = ["department_id", "department_name"]

In [15]:
emp_data = [(1,"Carlos", 17), (1,"Bob", 30), (2,"Jasmin", 26)]
emp_schema = ["department_id","employee_name", "age"]

In [16]:
df_emp = rdd_to_dataframe(emp_data, emp_schema)
df_dept = rdd_to_dataframe(dept_data, dept_schema)

In [17]:
df_dept.show()

+-------------+---------------+
|department_id|department_name|
+-------------+---------------+
|            1|       Big Data|
|            2|        Finance|
|            3|      Marketing|
+-------------+---------------+



In [18]:
spark=SparkSession.builder.appName("RDDToDataFrame").getOrCreate()

In [19]:
df_emp.createOrReplaceTempView('employees')
df_dept.createOrReplaceTempView('departments')

In [20]:
spark.sql('''select emp.*,dp.* from employees as emp join departments as dp on(emp.department_id=dp.department_id) where age>=18''').show()

+-------------+-------------+---+-------------+---------------+
|department_id|employee_name|age|department_id|department_name|
+-------------+-------------+---+-------------+---------------+
|            1|          Bob| 30|            1|       Big Data|
|            2|       Jasmin| 26|            2|        Finance|
+-------------+-------------+---+-------------+---------------+



In [10]:
spark.sql('''select emp.employee_name,emp.age,emp.department_id,dp.department_name from employees as emp join departments as dp on (emp.department_id=dp.department_id)''').createOrReplaceTempView('dp_employees')

In [11]:
spark.sql('''select * from dp_employees''').show()

+-------------+---+-------------+---------------+
|employee_name|age|department_id|department_name|
+-------------+---+-------------+---------------+
|       Carlos| 17|            1|       Big Data|
|          Bob| 30|            1|       Big Data|
|       Jasmin| 26|            2|        Finance|
+-------------+---+-------------+---------------+



In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as sqlfn
from pyspark.sql import types as sqlt

In [2]:
spark = SparkSession.builder.appName("RDDToDataFrame").getOrCreate()

In [3]:
spark = SparkSession.builder.appName('SparkStreamingLab1').master('local[2]').getOrCreate()

In [4]:
df_budgets = spark.read.option("multiline","true").json('C:\Windows\TEMP\Rar$DIa3316.41882\department_budgets.json')

In [5]:
df_budgets.printSchema()

root
 |-- budget: long (nullable = true)
 |-- budget_authorizer: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- cto: struct (nullable = true)
 |    |    |    |-- last_name: string (nullable = true)
 |    |    |    |-- name: string (nullable = true)
 |    |    |    |-- office: string (nullable = true)
 |-- budget_period: string (nullable = true)
 |-- department_id: long (nullable = true)
 |-- offices: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- cost_center: struct (nullable = true)
 |    |    |    |-- budget_status: string (nullable = true)
 |    |    |    |-- office: string (nullable = true)



In [26]:
df_budgets.select('offices').where('department_id == 1').show(truncate=False)

+-------------------------------------------------------------------------+
|offices                                                                  |
+-------------------------------------------------------------------------+
|[{{denied, new york}}, {{approved, mumbai}}, {{approved, san francisco}}]|
+-------------------------------------------------------------------------+



In [29]:
df_budgets.createOrReplaceTempView('budgets_json')

In [ ]:
spark.sql('''
          select  emp.employee_name, 
                  emp.department_id, 
                  bud.budget, 
                  bud.budget_period, 
                  bud.offices[0].cost_center.office as office_1,
                  bud.offices[0].cost_center.budget_status as budget_status_1,
                  bud.offices[1].cost_center.office as office_2,
                  bud.offices[1].cost_center.budget_status as budget_status_2,
                  bud.offices[2].cost_center.office as office_3,
                  bud.offices[2].cost_center.budget_status as budget_status_3,
                  nvl(bud.budget_authorizer[0].cto.name,"no CTO registered")  as cto_name,
                  nvl(bud.budget_authorizer[0].cto.last_name,"no CTO registered") as cto_last_name
          from dept_employees as emp
            inner join budgets_json as bud on (emp.department_id = bud.department_id)
          ''').show()

In [31]:
schema_employee = sqlt.StructType([
    sqlt.StructField('employee_id',sqlt.IntegerType(), True),
    sqlt.StructField('department_name',sqlt.StringType(), True),
    sqlt.StructField('name',sqlt.StringType(), True),
    sqlt.StructField('last_name',sqlt.StringType(), True),
    sqlt.StructField('hire_timestamp',sqlt.TimestampType(), True)
])

In [ ]:
df_employees = spark.readStream.format('csv').schema(schema_employee)\
                    .option('header',True)\
                    .option('maxFilesPerTrigger',1)\
                    .load(r'datasets/csv/')

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.master('local[1]').appName('spark-version-x').getOrCreate()

In [4]:
input_path=r'C:\Users\Yunis\production_data\yellow_taxi\yellow_tripdata_2024-01.parquet'
output_path=r'C:\Users\Yunis\production_data\blue_taxi'


In [5]:
df=spark.read.parquet(input_path)

In [6]:
df.createOrReplaceTempView('yellow_taxi')

In [7]:
df_output=spark.sql('''select VendorID,tpep_pickup_datetime,passenger_count from yellow_taxi where total_amount>1''')

In [11]:
df_output.write.mode('overwrite').parquet(output_path)

Py4JJavaError: An error occurred while calling o43.parquet.
: java.lang.UnsatisfiedLinkError: 'boolean org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(java.lang.String, int)'
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access0(Native Method)
	at org.apache.hadoop.io.nativeio.NativeIO$Windows.access(NativeIO.java:793)
	at org.apache.hadoop.fs.FileUtil.canRead(FileUtil.java:1249)
	at org.apache.hadoop.fs.FileUtil.list(FileUtil.java:1454)
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:601)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.getAllCommittedTaskPaths(Fil

In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType,StructType,StructField

In [10]:
spark= SparkSession.builder.master("local[1]").appName("spark_app").getOrCreate()

In [11]:
file=r'C:\Users\Yunis\Desktop\yellow_tripdata_2024-01 (1).parquet'
df=spark.read.parquet(file)

In [12]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [13]:
df2=df.select('*').show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-01-01 00:57:55|  2024-01-01 01:17:43|              1|         1.72|         1|                 N|         186|          79|           2|       17.7|  1.0|    0.5|       0.

In [14]:
df.createOrReplaceTempView('taxi_table')

In [15]:
spark.sql('select * from taxi_table where total_amount>1').show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-01-01 00:57:55|  2024-01-01 01:17:43|              1|         1.72|         1|                 N|         186|          79|           2|       17.7|  1.0|    0.5|       0.

In [16]:
spark.sql('select extract(year from tpep_pickup_datetime),* from taxi_table').show()

+---------------------------------------+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|extract(year FROM tpep_pickup_datetime)|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+---------------------------------------+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|                                   2024|       2| 2024-01-01 00:57

In [17]:
from pyspark.sql.functions import to_date,col
df_final=df.withColumn("p_date",to_date(col('tpep_pickup_datetime')))

In [18]:
df_final.select('*').show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|    p_date|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+----------+
|       2| 2024-01-01 00:57:55|  2024-01-01 01:17:43|              1|         1.72|         1|                 N|         186|          79|           2|  

In [19]:
df_final.write.partitionBy('p_date').mode('overwrite').parquet(r"C:\Users\Yunis\Desktop\yellow_tripdata_2024-01.parquet")

Py4JJavaError: An error occurred while calling o62.parquet.
: java.lang.RuntimeException: java.io.FileNotFoundException: Hadoop bin directory does not exist: C:\Users\Yunis\Desktop\Hadoop\hadoop-3.4.0\bin\bin -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLoca

In [12]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [12]:
data = [("1", "Standard rate"), ("2", "JFK"), ("3", "Newark"),("4","Nassau or Westchester "),("5","Negotiated fare"), ("6","Group ride")]

In [13]:
from pyspark.sql.types import StructType, StructField, StringType
schema=StructType([StructField("rate_code_id",StringType(),True),StructField("area_name",StringType(),True)])

In [14]:
df_rate_code=spark.createDataFrame(data=data,schema=schema)

In [15]:
df_rate_code.show()

+------------+--------------------+
|rate_code_id|           area_name|
+------------+--------------------+
|           1|       Standard rate|
|           2|                 JFK|
|           3|              Newark|
|           4|Nassau or Westche...|
|           5|     Negotiated fare|
|           6|          Group ride|
+------------+--------------------+



In [16]:
df.join(df_rate_code,df["RatecodeID"]==df_rate_code["rate_code_id"],"inner").show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------+---------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|rate_code_id|      area_name|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+------------+---------------+
|       1| 2024-01-01 00:42:05|  2024-01-01 01:16:49|              1|         23.9|         5|      

In [18]:
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [19]:
df_na_rate_codes=df.na.fill(value=0,subset=["RatecodeID"])

In [20]:
from pyspark.sql.functions import count
df_na_rate_codes.where("RatecodeID=0").select(count("*").alias("numb")).show()

+------+
|  numb|
+------+
|140162|
+------+



In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date,year

In [2]:
spark=SparkSession.builder.master("local[1]").appName("yun").getOrCreate()

In [3]:
local_file=r'C:\Users\Yunis\Desktop\yellow_tripdata_2024-01 (1).parquet'
df_taxi=spark.read.parquet(local_file)

In [4]:
df_taxi.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2024-01-01 00:57:55|  2024-01-01 01:17:43|              1|         1.72|         1|                 N|         186|          79|           2|       17.7|  1.0|    0.5|       0.

In [5]:
df_taxi=df_taxi.where(year(col("tpep_pickup_datetime"))=='2023')

In [6]:
df_taxi.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2023-12-31 23:56:46|  2024-01-01 00:12:06|              2|         2.38|         1|                 N|         236|         142|           1|       15.6|  1.0|    0.5|       1.

In [8]:
#df_final.select('p_date').show()

In [10]:
#df_final.printSchema()

In [11]:
df_taxi.createOrReplaceTempView("taxi_table")

In [23]:
df_final.createOrReplaceGlobalTempView("temp_table")

In [12]:
#qla=spark.sql("select avg(trip_distance) from taxi_table where p_date='2024-02-14' and RatecodeID=2")
qla=spark.sql("select * from taxi_table").show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2023-12-31 23:56:46|  2024-01-01 00:12:06|              2|         2.38|         1|                 N|         236|         142|           1|       15.6|  1.0|    0.5|       1.

In [14]:
qza=spark.sql("select * from taxi_table where passenger_count>2").show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|Airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       2| 2023-12-31 23:58:35|  2024-01-01 00:13:06|              6|         8.39|         1|                 N|         138|         217|           2|       33.1|  6.0|    0.5|       0.

In [22]:
df_taxi.explain(extended=True)

== Parsed Logical Plan ==
'Filter (year('tpep_pickup_datetime) = 2023)
+- Relation [VendorID#0,tpep_pickup_datetime#1,tpep_dropoff_datetime#2,passenger_count#3L,trip_distance#4,RatecodeID#5L,store_and_fwd_flag#6,PULocationID#7,DOLocationID#8,payment_type#9L,fare_amount#10,extra#11,mta_tax#12,tip_amount#13,tolls_amount#14,improvement_surcharge#15,total_amount#16,congestion_surcharge#17,Airport_fee#18] parquet

== Analyzed Logical Plan ==
VendorID: int, tpep_pickup_datetime: timestamp_ntz, tpep_dropoff_datetime: timestamp_ntz, passenger_count: bigint, trip_distance: double, RatecodeID: bigint, store_and_fwd_flag: string, PULocationID: int, DOLocationID: int, payment_type: bigint, fare_amount: double, extra: double, mta_tax: double, tip_amount: double, tolls_amount: double, improvement_surcharge: double, total_amount: double, congestion_surcharge: double, Airport_fee: double
Filter (year(cast(tpep_pickup_datetime#1 as date)) = cast(2023 as int))
+- Relation [VendorID#0,tpep_pickup_datetim

In [23]:
df_taxi.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)



In [24]:
spark.stop()

In [28]:
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, pandas_udf, PandasUDFType
from pyspark.sql.types import IntegerType

In [29]:
spark=SparkSession.builder.master("local[1]").appName("yanis").getOrCreate()